In [36]:
pip install cobra



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Libraries needed for importing the model and defining the model

In [37]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iJO1366.xml')

In [38]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [39]:
model.optimize() #initial optimal solution is 0.982

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,0.000000e+00
RNTR2c2,0.026541,2.775558e-17


Defining the metabolites

In [40]:
ethylene = Metabolite(id = 'eth_c', compartment='c', name='ethylene', formula='C2H4', charge=0)
ethylene_e = Metabolite(id = 'eth_e', compartment='e', name='ethylene_e', formula='C2H4', charge=0)
hydroxylarginine = Metabolite(id = 'hlarg_c', compartment='c', name='5-hydroxy-l-arginine', formula='C6H15N4O3', charge=1)
guanidine = Metabolite(id = 'guan_c', compartment='c', name='guanidine', formula='C1H6N3', charge=1)
ldelta1pyrroline5carboxylate = Metabolite(id = 'ld1p5carb_c', compartment='c', name='l-delta-1-pyrroline-5-carboxylate', formula='C5H7N1O2', charge=0)

Defining the reactions

In [41]:
Ethylene_forming_enzyme_main = Reaction('EFE_m')
Ethylene_forming_enzyme_side = Reaction('EFE_s')
Non_enzymatic_hlarg_degradation = Reaction('neHLARGdeg')
Ethylene_transport = Reaction('eth_tp')

Adding the metabolites to the reactions

In [42]:
Ethylene_transport.add_metabolites({ethylene: -1,
                            ethylene_e: 1,                      
                            })
Ethylene_forming_enzyme_side.add_metabolites({model.metabolites.akg_c: -1,              
                              model.metabolites.o2_c: -1,
                              model.metabolites.arg__L_c: -1,
                              hydroxylarginine: 1,
                              model.metabolites.co2_c: 1,
                              model.metabolites.succ_c: 1,
                             })

Non_enzymatic_hlarg_degradation.add_metabolites({hydroxylarginine: -1,
                              ldelta1pyrroline5carboxylate: 1,
                              guanidine: 1,
                              model.metabolites.h2o_c: 1,
                             })

Ethylene_forming_enzyme_main.add_metabolites({model.metabolites.akg_c: -1,              
                              model.metabolites.o2_c: -0.5,
                              #model.metabolites.arg__L_c: -1,  #in theory L-Arg is part of the reaction but is not changed              
                              ethylene: 1,
                              model.metabolites.co2_c: 3,
                              #model.metabolites.arg__L_c: 1,  #however, model does not accept L-arg as input and as output...
                              #model.metabolites.o2_c: 1,
                             })

Adding the reactions to the model

In [43]:
model.add_reactions([Ethylene_forming_enzyme_main])
model.add_reactions([Ethylene_forming_enzyme_side])
model.add_reactions([Non_enzymatic_hlarg_degradation])
model.add_reactions([Ethylene_transport])

Setting Reactions as demand reaction:
There are three types of reactions: Exchanges, Sinks and Demands
An exchange reaction is a reversible reaction that adds to or removes an extracellular metabolite from the extracellular compartment. A demand reaction is an irreversible reaction that consumes an intracellular metabolite. A sink is similar to an exchange but specifically for intracellular metabolites, i.e., a reversible reaction that adds or removes an intracellular metabolite.

In [44]:
model.add_boundary(model.metabolites.eth_c, type='demand')
model.add_boundary(model.metabolites.eth_e, type='exchange')
model.add_boundary(model.metabolites.hlarg_c, type='demand')
model.add_boundary(model.metabolites.guan_c, type='demand')
model.add_boundary(model.metabolites.ld1p5carb_c, type='demand')

Reaction identifier,DM_ld1p5carb_c
Name,l-delta-1-pyrroline-5-carboxylate demand
Memory address,0x7fd88862d330
Stoichiometry,ld1p5carb_c --> l-delta-1-pyrroline-5-carboxylate -->
GPR,
Lower bound,0
Upper bound,1000.0


Adding gene association:
AT1G05010 = locus tag found here: https://www.ncbi.nlm.nih.gov/gene?term=ethylene%20forming%20enzyme[gene]%20AND%20alive[prop]%20NOT%20newentry[gene]%20&sort=weight

In [45]:
Ethylene_forming_enzyme_main.gene_reaction_rule = '( AT1G05010 )'
Ethylene_forming_enzyme_main.genes

Ethylene_forming_enzyme_side.gene_reaction_rule = '( AT1G05010 )'
Ethylene_forming_enzyme_side.genes

frozenset({<Gene AT1G05010 at 0x7fd8858e5a50>})

Itterate through model to check for successful implementation:

In [46]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
EX_cm_e : cm_e --> 
EX_cmp_e : cmp_e --> 
EX_co2_e : co2_e <=> 
EX_cobalt2_e : cobalt2_e <=> 
DM_4crsol_c : 4crsol_c --> 
DM_5drib_c : 5drib_c --> 
DM_aacald_c : aacald_c --> 
DM_amob_c : amob_c --> 
DM_mththf_c : mththf_c --> 
EX_colipa_e : colipa_e --> 
DM_oxam_c : oxam_c --> 
EX_glc__D_e : glc__D_e <=> 
EX_glcn_e : glcn_e --> 
BIOMASS_Ec_iJO1366_WT_53p95M : 0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 54.119975 atp_c + 0.000116 bmocogdp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c +

In [47]:
model.optimize() #to check if adding metabolites and reactions had an effect on biomass -> it hasnt, good

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-0.296557
EX_co2_e,19.675223,0.000000
EX_cobalt2_e,-0.000025,0.000000
DM_4crsol_c,0.000219,0.000000
...,...,...
DM_eth_c,0.000000,0.000000
EX_eth_e,0.000000,0.000000
DM_hlarg_c,0.000000,0.000000
DM_guan_c,0.000000,0.000000


Checking the charge:

In [48]:
#print("Charge:")
#print("arg__L_c:")
#print(model.metabolites.arg__L_c.charge)
#print("succ_c:")
#print(model.metabolites.succ_c.charge)
#print("akg_c:")
#print(model.metabolites.akg_c.charge)
#print("O2_c:")
#print(model.metabolites.o2_c.charge)
#print("co2_c:")
#print(model.metabolites.co2_c.charge)
#print("hlarg_c:")
#print(model.metabolites.hlarg_c.charge)
#print("eth_c:")
#print(model.metabolites.eth_c.charge)

Create a  xml file with the modified model

In [49]:
from cobra.io import write_sbml_model

In [50]:
write_sbml_model(model, "modified_model.xml")